<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/04_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [3]:
zip_ref=zipfile.ZipFile('10_food_classes_10_percent.zip')

In [4]:
zip_ref.extractall()
zip_ref.close()

In [6]:
import os

In [7]:
for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f'There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}')

There are 2 directories and 0 images in 10_food_classes_10_percent
There are 10 directories and 0 images in 10_food_classes_10_percent/test
There are 0 directories and 250 images in 10_food_classes_10_percent/test/grilled_salmon
There are 0 directories and 250 images in 10_food_classes_10_percent/test/ramen
There are 0 directories and 250 images in 10_food_classes_10_percent/test/chicken_curry
There are 0 directories and 250 images in 10_food_classes_10_percent/test/pizza
There are 0 directories and 250 images in 10_food_classes_10_percent/test/sushi
There are 0 directories and 250 images in 10_food_classes_10_percent/test/fried_rice
There are 0 directories and 250 images in 10_food_classes_10_percent/test/hamburger
There are 0 directories and 250 images in 10_food_classes_10_percent/test/ice_cream
There are 0 directories and 250 images in 10_food_classes_10_percent/test/steak
There are 0 directories and 250 images in 10_food_classes_10_percent/test/chicken_wings
There are 10 directori

In [5]:
train_dir='10_food_classes_10_percent/train'
test_dir='10_food_classes_10_percent/test'

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen=ImageDataGenerator(rescale=(1/255))
test_datagen=ImageDataGenerator(rescale=(1/255))

In [11]:
print('Training data : ')
train_data=train_datagen.flow_from_directory(train_dir,batch_size=32,target_size=(256,256),class_mode='categorical')
print('Testing data : ')
test_data=test_datagen.flow_from_directory(test_dir,target_size=(256,256),batch_size=32,class_mode='categorical')


Training data : 
Found 750 images belonging to 10 classes.
Testing data : 
Found 2500 images belonging to 10 classes.


In [13]:
import datetime
import tensorflow as tf

In [15]:

def create_tensorboard_callback(dirname,experiment_name):
    log_dir=dirname + "/" + experiment_name + datetime.datetime.now().strftime("%y%m%d-%H%M%S")
    tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f'saving log dir to {log_dir}')
    return tensorboard_callback

Resnet v2-50  and Efficeient net

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [18]:
# we will compare two models resnetv50 and efficentnet
resnet_url="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficientnet_url="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [19]:
# Lets functionize model creation

def create_model(model_url,num_classes=10):
    feature_extractor_layer=hub.KerasLayer(model_url,trainable=False,name="Feature_etxractor_layer",input_shape=(256,256,3))

    model=tf.keras.Sequential(
        feature_extractor_layer,
        layers.Dense(num_classes,activation='softmax',name='output layer')
    )
    return model